# Irrigation System Hyperparameter Optimization Study

## Overview
This notebook implements a comprehensive Bayesian hyperparameter optimization study for a safe reinforcement learning irrigation system. The study focuses on minimizing constraint violation costs while maintaining system safety.

## Study Objectives
1. **Primary Goal**: Minimize irrigation constraint violation costs
2. **Secondary Goal**: Identify most important hyperparameters for performance
3. **Safety Goal**: Maintain safe operation under all parameter configurations

## Methodology
- **Optimizer**: Optuna with Tree-structured Parzen Estimator (TPE)
- **Search Strategy**: Progressive multi-phase optimization
- **Safety Features**: Checkpointing, memory management, error recovery
- **Convergence Detection**: Automatic stopping based on coefficient of variation

## Expected Runtime
- **Total Trials**: 50-125 (adaptive based on convergence)
- **Time per Trial**: ~10-15 minutes (25 epochs each)
- **Total Duration**: 8-30 hours depending on convergence

In [ ]:
# === IMPORTS AND SETUP ===
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from optuna.samplers import TPESampler
import gc
import torch
import pickle
from datetime import datetime

# Import custom modules
from water_environment import WaterEnvironment
from training import train_agent

# Display settings
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print(f"Study started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Optuna version: {optuna.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# === DATA SETUP AND VALIDATION ===

# Directory configuration
folder_path_initial = '/home/egomez/irrigation_project/'
folder_path_output = '/scratch/egomez/irrigation_project_output/'
data_file = os.path.join(folder_path_initial, 'daily_weather_data.csv')
model_directory = os.path.join(folder_path_output, 'models')

# Create necessary directories
os.makedirs(model_directory, exist_ok=True)
os.makedirs(os.path.join(model_directory, 'optimization'), exist_ok=True)
os.makedirs(os.path.join(model_directory, 'test'), exist_ok=True)

df = pd.read_csv(data_file)

# Study configuration
STUDY_CONFIG = {
    'max_total_trials': 125,
    'phase1_trials': 50,    # Initial exploration
    'phase2_trials': 50,    # Refinement  
    'phase3_trials': 25,    # Final optimization
    'convergence_threshold': 0.05,  # CV threshold for stopping
    'checkpoint_frequency': 10,
    'memory_cleanup_frequency': 5,
}

print(f"\nStudy Configuration:")
for key, value in STUDY_CONFIG.items():
    print(f"  {key}: {value}")

print(f"\nOutput directories:")
print(f"  Models: {model_directory}")
print(f"  Optimization: {os.path.join(model_directory, 'optimization')}")
print(f"  Test: {os.path.join(model_directory, 'test')}")

In [ ]:
base_env_params = {
    'weather_data': df, 
    'n_days_ahead': 7
}

base_agent_params = {
    'polyak':          0.999,
    'gamma':           0.99,
    'model_type':      'DDPGLagrangian',
    'hidden_layers':   [256, 256],
    'chance_const':    0.95,
    'actor_lr':        1e-5,  # Will be optimized
    'critic_lr':       1e-4,  # Will be optimized
    'cost_critic_lr':  1e-3,  # Will be optimized
    'KP':              0.01,  # Will be optimized
    'KI':              0.01,  # Will be optimized
    'KD':              0.0,   # Will be optimized
    'safe_buffer':     True,
}

base_training_params = {
    'num_epochs': 500,  # Reduced during optimization
    'sample_episode_num': 20,
    'episode_rerun_num': 20,
    'evaluate_episode_num': 15,
    'plot_save_frequency': 100,
    'max_episode_steps': 1000,
    'batch_size': 512,
    'seed': 0,
    'device': "gpu",
    'threads': 1,
    'reset_epochs': {100, 200, 300, 400}
}

In [ ]:
# === STUDY UTILITIES AND VALIDATION ===

def validate_setup():
    """Validate that all components are properly configured."""
    print("=== SETUP VALIDATION ===")
    
    validation_results = {
        'data_file': os.path.exists(data_file),
        'output_dir': os.path.exists(model_directory),
        'gpu_available': torch.cuda.is_available() if 'gpu' in base_training_params['device'] else True,
        'modules_importable': True  # Already imported successfully
    }
    
    print("Component Status:")
    for component, status in validation_results.items():
        status_icon = "✅" if status else "❌"
        print(f"  {component}: {status_icon}")
    
    # Check disk space
    try:
        import shutil
        total, used, free = shutil.disk_usage(folder_path_output)
        free_gb = free / (1024**3)
        print(f"  disk_space: ✅ ({free_gb:.1f} GB free)")
        if free_gb < 10:
            print("    ⚠️  Warning: Less than 10GB free space available")
    except:
        print(f"  disk_space: ⚠️  (Could not check)")
    
    # Check memory
    if torch.cuda.is_available():
        try:
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            print(f"  gpu_memory: ✅ ({gpu_memory:.1f} GB)")
        except:
            print(f"  gpu_memory: ⚠️  (Could not check)")
    
    all_valid = all(validation_results.values())
    print(f"\nOverall Status: {'✅ Ready for optimization' if all_valid else '❌ Issues detected'}")
    
    return all_valid

def cleanup_memory():
    """Clean up GPU and system memory."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def get_study_status():
    """Get current study progress."""
    study_file = os.path.join(model_directory, 'study_checkpoint.pkl')
    try:
        with open(study_file, 'rb') as f:
            study = pickle.load(f)
        
        completed_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
        failed_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])
        
        print(f"=== STUDY STATUS ===")
        print(f"Total trials: {len(study.trials)}")
        print(f"Completed: {completed_trials}")
        print(f"Failed: {failed_trials}")
        
        if study.best_trial:
            print(f"Best cost: {study.best_value:.4f}")
            print(f"Best trial: #{study.best_trial.number}")
        
        return study, completed_trials, failed_trials
    except FileNotFoundError:
        print("=== STUDY STATUS ===")
        print("No existing study found - will start fresh")
        return None, 0, 0

def estimate_remaining_time(completed_trials, target_trials, avg_time_per_trial=12):
    """Estimate remaining optimization time."""
    remaining_trials = max(0, target_trials - completed_trials)
    remaining_minutes = remaining_trials * avg_time_per_trial
    remaining_hours = remaining_minutes / 60
    
    print(f"\n=== TIME ESTIMATION ===")
    print(f"Completed trials: {completed_trials}")
    print(f"Target trials: {target_trials}")
    print(f"Remaining trials: {remaining_trials}")
    print(f"Estimated time remaining: {remaining_hours:.1f} hours ({remaining_minutes:.0f} minutes)")
    
    if remaining_hours > 24:
        print(f"  ≈ {remaining_hours/24:.1f} days")
    
    return remaining_hours

# Run validation
validation_passed = validate_setup()

# Check existing study status
existing_study, completed_trials, failed_trials = get_study_status()

# Estimate time for full study
target_trials = STUDY_CONFIG['max_total_trials']
if existing_study:
    remaining_time = estimate_remaining_time(completed_trials, target_trials)
else:
    print(f"\n=== TIME ESTIMATION ===")
    print(f"New study - estimated total time: {target_trials * 12 / 60:.1f} hours")

In [ ]:
def objective(trial):
    """Optuna objective function for Bayesian hyperparameter optimization."""
    import gc
    import torch
    
    # Clear GPU memory at start of each trial
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    # Sample hyperparameters
    polyak = trial.suggest_float('polyak', 0.9, 0.999)
    actor_lr = trial.suggest_float('actor_lr', 1e-6, 1e-3, log=True)
    critic_lr = trial.suggest_float('critic_lr', 1e-5, 1e-3, log=True)
    cost_critic_lr = trial.suggest_float('cost_critic_lr', 1e-5, 1e-2, log=True)
    kp = trial.suggest_float('KP', 0.001, 10, log=True)
    ki = trial.suggest_float('KI', 0.001, 1, log=True)
    kd = trial.suggest_float('KD', 0.0, 0.01)
    
    # Create agent parameters with sampled values
    agent_params = base_agent_params.copy()
    agent_params.update({
        'polyak': polyak,
        'actor_lr': actor_lr,
        'critic_lr': critic_lr,
        'cost_critic_lr': cost_critic_lr,
        'KP': kp,
        'KI': ki,
        'KD': kd,
    })
    
    # Create shorter training for optimization (faster trials)
    training_params = base_training_params.copy()
    training_params.update({
        'num_epochs': 25,  # Shorter for stability
        'plot_save_frequency': 1000,  # Disable plotting during optimization
        'evaluate_episode_num': 1  # Reduce evaluation episodes
    })
    
    # Create unique identifier for this trial
    identifier = f"trial_{trial.number}_actor{actor_lr:.1e}_critic{critic_lr:.1e}_kp{kp:.3f}"
    exp_dir = os.path.join(model_directory, 'optimization', identifier)
    os.makedirs(exp_dir, exist_ok=True)
    
    try:
        print(f"Starting trial {trial.number}...")
        
        # Create environment
        env = WaterEnvironment(**base_env_params)
        agent_params.update({
            'env': env,
            'chkpt_dir': exp_dir,
        })
        
        # Train agent and get performance metrics
        results = train_agent(base_env_params, agent_params, training_params, exp_dir, identifier)
        
        # Extract cost metrics
        total_costs = results['total_costs']
        avg_cost = results['avg_cost_per_episode']
        
        # For cost minimization, return negative cost (since Optuna maximizes)
        objective_value = -avg_cost
        
        # Log trial results
        print(f"Trial {trial.number} completed: avg_cost={avg_cost:.3f}, total_costs={total_costs:.3f}")
        
        # Clean up
        del env, results
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
        return objective_value
        
    except Exception as e:
        print(f"Trial {trial.number} failed with error: {e}")
        print(f"Trial parameters: {trial.params}")
        
        # Clean up on error
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
        return -1000  # Return very poor score for failed trials

def run_bayesian_optimization(n_trials=10, save_every=5):
    """Run Bayesian optimization with checkpointing."""
    import pickle
    
    study_file = os.path.join(model_directory, 'study_checkpoint.pkl')
    
    # Try to load existing study
    try:
        with open(study_file, 'rb') as f:
            study = pickle.load(f)
        print(f"Loaded existing study with {len(study.trials)} trials")
        start_trial = len(study.trials)
    except FileNotFoundError:
        # Create new study
        study = optuna.create_study(
            direction='maximize',
            sampler=TPESampler(seed=42),
            study_name='irrigation_cost_minimization'
        )
        start_trial = 0
        print("Created new optimization study")
    
    print(f"Running {n_trials} trials (starting from trial {start_trial})...")
    print("Objective: Minimize constraint violation costs")
    
    try:
        for i in range(n_trials):
            print(f"\n--- Running trial {start_trial + i + 1}/{start_trial + n_trials} ---")
            
            # Run single trial
            study.optimize(objective, n_trials=1)
            
            # Save checkpoint every few trials
            if (i + 1) % save_every == 0 or i == n_trials - 1:
                with open(study_file, 'wb') as f:
                    pickle.dump(study, f)
                print(f"Checkpoint saved after {len(study.trials)} trials")
    
    except KeyboardInterrupt:
        print("Optimization interrupted by user")
    except Exception as e:
        print(f"Optimization failed: {e}")
    
    # Save final study
    with open(study_file, 'wb') as f:
        pickle.dump(study, f)
    
    # Print results
    if study.trials:
        print("\nOptimization completed!")
        print(f"Total trials: {len(study.trials)}")
        print(f"Best trial: {study.best_trial.number}")
        print(f"Best objective value: {study.best_value:.3f}")
        print(f"Best average cost: {-study.best_value:.3f}")
        print("\nBest parameters:")
        for key, value in study.best_params.items():
            print(f"  {key}: {value}")
    
    return study

# Safe usage - start with fewer trials
# study = run_bayesian_optimization(n_trials=5)

In [ ]:
def load_study_checkpoint():
    """Load saved study from checkpoint."""
    import pickle
    study_file = os.path.join(model_directory, 'study_checkpoint.pkl')
    try:
        with open(study_file, 'rb') as f:
            study = pickle.load(f)
        print(f"Loaded study with {len(study.trials)} trials")
        return study
    except FileNotFoundError:
        print("No checkpoint found")
        return None

def analyze_optimization_results(study):
    """Analyze and visualize optimization results for cost minimization."""
    
    import matplotlib.pyplot as plt

    if not study or not study.trials:
        print("No study data to analyze")
        return None
    
    # Get cost values directly (no conversion needed since we minimize costs directly)
    trials = study.trials
    cost_values = [t.value for t in trials if t.value is not None and t.state == optuna.trial.TrialState.COMPLETE]
    
    if not cost_values:
        print("No completed trials to analyze")
        return None
    
    # Plot optimization history
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Cost minimization history
    axes[0, 0].plot(cost_values)
    axes[0, 0].set_title('Cost Minimization Progress')
    axes[0, 0].set_xlabel('Trial')
    axes[0, 0].set_ylabel('Average Cost per Episode')
    axes[0, 0].grid(True, alpha=0.3)
    
    if study.best_value is not None:
        best_cost = study.best_value  # Direct cost value (no conversion)
        axes[0, 0].axhline(best_cost, color='red', linestyle='--', 
                          label=f'Best Cost: {best_cost:.3f}')
        axes[0, 0].legend()
    
    # 2. Parameter importance
    try:
        if len(cost_values) > 3:  # Need multiple trials for importance
            importance = optuna.importance.get_param_importances(study)
            params = list(importance.keys())
            importances = list(importance.values())
            
            axes[0, 1].barh(params, importances)
            axes[0, 1].set_title('Parameter Importance')
            axes[0, 1].set_xlabel('Importance')
        else:
            axes[0, 1].text(0.5, 0.5, 'Need more trials\nfor importance analysis', 
                            ha='center', va='center', transform=axes[0, 1].transAxes)
    except:
        axes[0, 1].text(0.5, 0.5, 'Parameter importance\nnot available', 
                        ha='center', va='center', transform=axes[0, 1].transAxes)
    
    # 3. Best parameters table
    if study.best_params:
        best_params = study.best_params
        param_names = list(best_params.keys())
        param_values = [f"{best_params[p]:.4f}" if isinstance(best_params[p], float) 
                       else str(best_params[p]) for p in param_names]
        
        axes[1, 0].axis('tight')
        axes[1, 0].axis('off')
        table = axes[1, 0].table(cellText=[[n, v] for n, v in zip(param_names, param_values)],
                                colLabels=['Parameter', 'Best Value'],
                                cellLoc='left',
                                loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        axes[1, 0].set_title(f'Best Parameters (Cost: {best_cost:.3f})')
    else:
        axes[1, 0].text(0.5, 0.5, 'No best parameters\navailable', 
                        ha='center', va='center', transform=axes[1, 0].transAxes)
    
    # 4. Distribution of costs
    if len(cost_values) > 1:
        axes[1, 1].hist(cost_values, bins=min(20, len(cost_values)), alpha=0.7, edgecolor='black')
        if study.best_value is not None:
            axes[1, 1].axvline(best_cost, color='red', linestyle='--', 
          )                   label=f'Best: {best_cost:.3f}')
            axes[1, 1].legend()
    else:
        axes[1, 1].text(0.5, 0.5, 'Need more trials\nfor distribution', 
                        ha='center', va='center', transform=axes[1, 1].transAxes)
    
    axes[1, 1].set_title('Distribution of Average Costs')
    axes[1, 1].set_xlabel('Average Cost per Episode')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    if study.best_value is not None:
        print("\n=== COST MINIMIZATION SUMMARY ===")
        print(f"Total trials: {len(study.trials)}")
        print(f"Completed trials: {len(cost_values)}")
        print(f"Best (minimum) cost: {best_cost:.4f}")
        if len(cost_values) > 1:
            print(f"Worst (maximum) cost: {max(cost_values):.4f}")
            print(f"Mean cost across trials: {np.mean(cost_values):.4f}")
            print(f"Std cost across trials: {np.std(cost_values):.4f}")
            print(f"Cost improvement: {max(cost_values) - best_cost:.4f}")
    
    return fig

def run_single_trial_test():
    """Test a single trial to check if setup works."""
    print("Running single trial test...")
    
    # Use default parameters for test
    test_params = base_agent_params.copy()
    test_training_params = base_training_params.copy()
    test_training_params.update({
        'num_epochs': 5,  # Very short test
        'plot_save_frequency': 1000
    })
    
    identifier = "single_trial_test"
    exp_dir = os.path.join(model_directory, 'test', identifier)
    os.makedirs(exp_dir, exist_ok=True)
    
    try:
        env = WaterEnvironment(**base_env_params)
        test_params.update({
            'env': env,
            'chkpt_dir': exp_dir,
        })
        
        results = train_agent(base_env_params, test_params, test_training_params, exp_dir, identifier)
        print(f"Test successful! Cost: {results['avg_cost_per_episode']:.3f}")
        return True
    except Exception as e:
        print(f"Test failed: {e}")
        return False

# Recovery tools - uncomment as needed:
# study = load_study_checkpoint()  # Load previous progress
# test_success = run_single_trial_test()  # Test basic functionality
# study = run_bayesian_optimization(n_trials=3)  # Start small
# analyze_optimization_results(study)  # Analyze results
    print("- Verify environment variables are set correctly")
    print("- Confirm network connectivity if required")
    

In [ ]:
# === EXECUTE COMPREHENSIVE STUDY ===

print("\n" + "="*60)
print("🎯 IRRIGATION HYPERPARAMETER OPTIMIZATION STUDY")
print("="*60)
print("Select your execution option by uncommenting ONE of the lines below:")
print("")

# === EXECUTION OPTIONS ===

# Option 1: Run the complete comprehensive study (RECOMMENDED)
# This will run the full multi-phase optimization with 50-125 trials
# Estimated time: 8-30 hours depending on convergence
print("Option 1: Complete Study (Recommended)")
print("  Uncomment: study = run_comprehensive_study()")
study = run_comprehensive_study()

print("")

# Option 2: Quick test and small optimization (TESTING)
# This will run a quick test followed by a small optimization
print("Option 2: Quick Test + Small Optimization")
print("  Uncomment the lines below:")
# test_success = run_single_trial_test()
# if test_success:
#     study = run_bayesian_optimization(n_trials=5)
#     analyze_optimization_results(study)

print("")

# Option 3: Individual components (DEBUGGING)
# Run specific components individually for testing/debugging
print("Option 3: Individual Components")
print("  Uncomment any of these lines:")
# test_success = run_single_trial_test()                    # Quick functionality test (2-3 minutes)
# study = run_bayesian_optimization(n_trials=10)            # Small optimization run (2-3 hours)
# study = load_study_checkpoint()                           # Load existing progress
# analyze_optimization_results(study)                       # Analyze current results

print("")

# Option 4: Resume existing study (if you have a checkpoint)
print("Option 4: Resume/Analyze Existing Study")
print("  Uncomment the lines below:")
# study = load_study_checkpoint()
# if study:
#     analyze_optimization_results(study)
#     print(f"\\nStudy has {len(study.trials)} trials. To continue:")
#     print("  study = run_bayesian_optimization(n_trials=25)")
# else:
#     print("No existing study found. Start with Option 1 or 2.")

print("")
print("="*60)
print("📖 QUICK START INSTRUCTIONS:")
print("1. For first-time users: Start with Option 2 (Quick Test)")
print("2. For full optimization: Use Option 1 (Complete Study)")
print("3. To resume previous work: Use Option 4")
print("")
print("⚠️  IMPORTANT: Only uncomment ONE option at a time!")
print("💡 TIP: Test with Option 2 first to verify everything works")
print("="*60)

# Show current system status
print(f"\n📊 CURRENT SYSTEM STATUS:")
print(f"  Device: {base_training_params['device'].upper()}")
print(f"  Validation: {'✅ PASSED' if validation_passed else '❌ FAILED'}")
if existing_study:
    print(f"  Existing trials: {len(existing_study.trials)} (best cost: {existing_study.best_value:.4f})")
else:
    print(f"  Existing trials: None (fresh start)")
print(f"  Ready to start: {'✅ YES' if validation_passed else '❌ NO (fix validation issues first)'}")